In [11]:
def processing_topic_and_h2(data):
    web = BeautifulSoup(data[2], 'html.parser')
    
    # Select Tags
    a_list = web.select('a')
    h2_list = web.select('h2')
    footer_list = web.select('footer')
    
    def get_topics():
        nonlocal a_list
        nonlocal footer_list
        topics = ''
        if a_list == None: return topics
        for i in range(len(a_list)):
            if not a_list[i].has_attr('href'): continue
                
            if "/category" in str(a_list[i]['href']) and a_list[i].text.strip() != '':
                topics = topics + a_list[i].text.strip() + ' '
                
        return topics[:-1]
    
    def get_h2():
        nonlocal h2_list
        h2 = ''
        if h2_list == None: return h2
        for i in range(len(h2_list)):
            if not h2_list[i].has_attr('class'): continue    
            
            if "title" in str(h2_list[i]['class']) and h2_list[i].text.strip() != '':
                h2 = h2 + h2_list[i].text.strip() + ' '
                
        return h2[:-1]
    
    topics = get_topics()
    h2 = get_h2()
    return [topics, h2]


In [1]:
import csv, sys
from bs4 import BeautifulSoup
from datetime import datetime
from joblib import Parallel, delayed
import re
dataset_dir = './'
train_csv = 'train.csv'
test_csv = 'test.csv'
clean_train_csv = 'clean.csv'
clean_test_csv = 'test_clean.csv'

#csv.field_size_limit(sys.maxsize)
maxInt = sys.maxsize
while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)
csvfile = open(dataset_dir + train_csv, newline='', encoding="utf-8")
test_csvfile = open(dataset_dir + test_csv, newline='', encoding="utf-8")

rows = csv.reader(csvfile, delimiter=',')
train_raw_data = [row for row in rows]

rows = csv.reader(test_csvfile, delimiter=',')
test_raw_data = [row for row in rows]

In [69]:
def processing(data):
    label = 0

    web = BeautifulSoup(data[2], 'html.parser')
    web2 = str(data[2])
    
    # Select Tags
    a_list = web.select('a')
    src_list = web.select('src')
    h1_list = web.select('h1')
    section_list = web.select('section')
    
    article_list = web.select('article')
    span = web.select_one('span')
    time = web.select_one('time')
    
    # Extract Method
    def get_title():
        nonlocal h1_list
        title = ''
        if h1_list == None: return title
        
        
        for i in range(len(h1_list)):
            if not h1_list[i].has_attr('class'): continue
            if 'title' in str(h1_list[i]['class']):
                title = h1_list[i].text
                break
                
        return title
    
    def get_time():
        nonlocal time
        isWeekday = 0
        if time == None: return isWeekday
        
        
        time = time.text.strip().split()
        if len(time) == 0:
            isWeekday = 0
            
        else:
            time = datetime.strptime(time[0], "%Y-%m-%d")
            isWeekday = 1 if time.weekday() >= 5 else 0
        
        return isWeekday
    
    def get_links():
        nonlocal a_list
        nonlocal src_list
        links = 0
        if a_list == None or src_list == None: return links
        
        
        for i in range(len(a_list)):
            if not a_list[i].has_attr('href'): continue
            if 'http' in str(a_list[i]['href']):
                links += 1

        for i in range(len(src_list)):
            if not src_list[i].has_attr('href'): continue
            if 'http' in str(src_list[i]['href']):
                links += 1
                
        return links

    def get_author():
        nonlocal a_list
        nonlocal span
        author = ''
        if a_list == None: return author
        
        
        for i in range(len(a_list)):
            if not a_list[i].has_attr('href'): continue
                
            if "author" in str(a_list[i]['href']) and a_list[i].text.strip() != '':
                author = a_list[i].text.strip()
                break

        if author in '' and span != None:
            author = span.text.strip()
        
        author = re.sub("[-:0-9]", "", author)
        author = author.replace('By ', '').replace(' UTC', '')
        return author
    
    def get_article():
        nonlocal section_list
        article = ''
        if section_list == None: return article

        
        for i in range(len(section_list)):
            if not section_list[i].has_attr('class'): continue

            if "article-content" in str(section_list[i]['class']):
                article = section_list[i].text.strip()
                break

        return article

    def get_channel():
        nonlocal article_list
        channel = ''
        if article_list == None:
            return channel
        
        for i in range(len(article_list)):
            if not article_list[i].has_attr('data-channel'): continue
            else:    
                channel = str(article_list[i]['data-channel'])
                break

        return channel
    
    def get_figure():
        figure = 0 
        figure = web2.count('.jpg') + web2.count('.jpeg') + web2.count('.png') + web2.count('.gif')
        return figure
    
    # Set Return Value
    isWeekday = get_time()
    title = get_title()
    links = get_links()
    author = get_author()
    article = get_article()
    channel = get_channel()
    figure = get_figure()
    
    
    # Change label
    label = 0 if int(data[1]) == -1 else int(data[1])
    return [label, isWeekday, title, links, author, article, channel, figure]

In [70]:
data = list(map(processing, train_raw_data[1:]))

In [71]:
import pandas as pd
df = pd.DataFrame(data, columns =['label', 'weekday', 'title', 'links', 'author', 'article', 'channel', 'figure'])
df.to_csv(dataset_dir + 'clean+figure.csv')

In [12]:
data = list(map(processing_topic, train_raw_data[1:10]))
print(data)

[['Asteroid Asteroids challenge Earth Space U.S. World', ''], ['patent-lawsuit theater Apps and Software Google open source opn pledge patent lawsuit theater software patents Tech U.S.', ''], ['NFL ESPN Entertainment NFL NFL Draft Sports Television', '1. Golf Cart Fail 2. Penalty Kick Fail 3. Mascot on Rollerblades Fail 4. Flopping Fail 5. Hockey Celebration Fail 6. Innocent Bystander Fail 7. Representing Your School Fail 8. Weightlifting Fail 9. Trampoline Dunk Fail 10. Hurdling Fail 11. Paying Attention Fail 12. Judgment Fail 13. Olympic Promo Fail 14. Grand Slam Celebration Fail 15. Slam Dunk Fail 16. High Dive Fail 17. Piggyback Ride Fail 18. Long Jump Fail 19. Touchdown Celebration Fail 20. Pro Golfer Fail 21. Head-First Slide Fail 22. Time Management Fail 23. Boogie Boarding Fail 24. Sportsmanship Fail 25. Standing on a Basketball Hoop Fail'], ['YouTube Sports Video Videos Watercooler', '1. Can You Say Banana? 2. Cuban Spider 3. Running Toy Poodle 4. Hokey Pokey Dog 5. Yay! Kermi

In [72]:
%matplotlib inline
import pandas as pd
import numpy as np
df = pd.read_csv('./clean+figure.csv')
print(df.head(5))
train = df.drop(['label', 'author', 'article', 'title'], axis='columns')
print(train.head(5))
from sklearn import preprocessing
labelencoder = preprocessing.LabelEncoder()
train['channel'] = labelencoder.fit_transform(df['channel'])
for i in range(len(train)):
    train.iloc[i, 0] = len(df.iloc[i, 6])
print(train.head(5))

   Unnamed: 0  label  weekday  \
0           0      0        0   
1           1      1        0   
2           2      1        0   
3           3      0        0   
4           4      0        0   

                                               title  links            author  \
0  NASA's Grand Challenge: Stop Asteroids From De...     14   Clara Moskowitz   
1  Google's New Open Source Patent Pledge: We Won...      8  Christina Warren   
2  Ballin': 2014 NFL Draft Picks Get to Choose Th...      4         Sam Laird   
3        Cameraperson Fails Deliver Slapstick Laughs      7         Sam Laird   
4  NFL Star Helps Young Fan Prove Friendship With...      9   Connor Finnegan   

                                             article        channel  figure  
0  There may be killer asteroids headed for Earth...          world       2  
1  Google took a stand of sorts against patent-la...           tech       3  
2  You've spend countless hours training to be an...  entertainment      28  
3 

In [74]:
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


SC = StandardScaler()
SC.fit(train)
X_std = SC.transform(train)

model = XGBClassifier(max_depth=5,learning_rate=0.015,n_estimators=100,objective="binary:logistic", eval_metric="error")
scores = cross_val_score(estimator=model, X=X_std, y=df['label'], \
                         cv=5, scoring='roc_auc')
print('%.3f (+/-%.3f)' % (scores.mean(), scores.std()))

D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The u

0.544 (+/-0.009)


In [46]:
print(labelencoder.classes_)

['advertising' 'apps-software' 'bus' 'business' 'comics' 'conversations'
 'dev-design' 'entertainment' 'film' 'gadgets' 'gaming' 'home' 'how-to'
 'howto' 'jobs' 'lifestyle' 'marketing' 'media' 'memes' 'mob' 'mobile'
 'music' 'pics' 'small-business' 'social-media' 'socmed' 'sports'
 'startups' 'tech' 'us' 'viral' 'watercooler' 'world']
